# Heart Disease Classifier – Final Project

## 1. Data Loading & Cleaning

## 2. Exploratory Data Analysis (EDA)

## 3. Unsupervised Learning: PCA + GMM

## 4. Supervised Learning: SVM, Random Forest, KNN

## 5. Feature Selection & Feature Importance

## 6. Evaluation: Accuracy, F1, Confusion Matrix


## 7. Conclusions & Discussion